# Project 1


Представьте, что вы работаете в компании, которая разрабатывает мобильные игры. 
К вам пришел менеджер с рядом задач по исследованию нескольких аспектов мобильного приложения:

В первую очередь, его интересует показатель retention. Напишите функцию для его подсчета. 

Помимо этого, в компании провели A/B тестирование наборов акционных предложений. На основе имеющихся данных определите, какой набор можно считать лучшим и на основе каких метрик стоит принять правильное решение. 

Предложите метрики для оценки результатов последнего прошедшего тематического события в игре.

### Задание 1

Retention – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока). Данные лежат в папке shared и имеют следующую структуру:


shared/problem1-reg_data.csv – данные о времени регистрации

reg_ts	uid
906166566	2
906344325	2
906686169	2
906893386	2
906980227	2

shared/problem1-auth_data.csv – данные о времени захода пользователей в игру

auth_ts	uid
906166566	2
924422172	3
937374732	4
947425117	5
955630339	6

Функция должна быть написана на python. В ходе решения можно тестировать работу функции как на полном датасете, так и на части (сэмпле) данных.

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as st
import pingouin as pg
from scipy.stats import chi2_contingency, chi2 

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from operator import attrgetter
from matplotlib import colors as mcolors

In [ ]:
#Открываем первый файл
df1 = pd.read_csv('~/shared/problem1-reg_data.csv', sep=';')
df1

In [ ]:
#Открываем второй файл
df2 = pd.read_csv('~/shared/problem1-auth_data.csv', sep=';')
df2

In [ ]:
#Соединим все таблицы в одну
df = df2.merge(df1, how='left', on='uid')
df.info()

In [ ]:
#Переводим время регистрации и время захода пользователей в игру в формат datetime64 
df['auth_ts'] = pd.to_datetime(df['auth_ts'], unit='s')
df['reg_ts'] = pd.to_datetime(df['reg_ts'], unit='s')


In [ ]:
#Извлекаем день из времени регистрации и времени захода пользователей в игру
df ['reg_date'] = df['reg_ts'].dt.to_period('D')
df ['auth_date'] = df['auth_ts'].dt.to_period('D')


In [ ]:
#Создаем отдельную колонку с количеством дней от времени регистрации
df['number_days'] = ((df.auth_date - df.reg_date)/ np.timedelta64(1, 'D'))


In [ ]:
#Агрегируем данные по когортам и рассчитанному количеству дней от времени регистрации
df_cohort = df.groupby(['reg_date', 'number_days']).agg({'uid':'nunique'}).reset_index()


In [ ]:
#Создаем сводную таблицу для когортного анализа 
cohort_pivot = df_cohort.pivot_table(index='reg_date', columns = 'number_days', values = 'uid')


In [ ]:
#Рассчитываем размеры когорт (первый столбец сводной таблицы)
cohort_size = cohort_pivot.iloc[:,0]


In [ ]:
#Вычисляем коэффициенты удержания, деля на размер когорты
retention_matrix = cohort_pivot.divide(cohort_size, axis =0)
retention_matrix

In [ ]:
with sns.axes_style("white"):
    fig, ax=plt.subplots(1,2, figsize=(16,12),sharey=True, gridspec_kw={'width_ratios':[1,11]})
    #Тепловая карта для коээфициентов удержания
    sns.heatmap(retention_matrix,
               mask=retention_matrix.isnull(),
               annot=True,
               fmt='.0%',
               cmap='RdYlGn',
               ax=ax[1])
    ax[1].set_title('Daily Retention', fontsize=16)
    ax[1].set(xlabel='Количество дней после регистрации ', ylabel='Дата регистрации')
    #Тепловая карта для размеров когорт
    cohort_size_df = pd.DataFrame(cohort_size).rename(columns={0:'cohort_size'})
    white_cmap=mcolors.ListedColormap(['white'])
    sns.heatmap(cohort_size_df,
               annot=True,
               cbar= False,
               fmt='g',
               cmap=white_cmap,
               ax=ax[0])
    fig.tight_layout()
    plt.show()

In [ ]:
#Создаем функцию
def retention(df1, df2, Beginning, End):
    df = df2.merge(df1, how='left', on='uid')
    Beginning = pd.to_datetime(Beginning, unit='s').dt.to_period('D')
    End = pd.to_datetime(End, unit='s').dt.to_period('D')
    df ['reg_date'] = df['reg_ts'].dt.to_period('D')
    df ['auth_date'] = df['auth_ts'].dt.to_period('D')
    #Добавим условие, чтобы мы могли использовать функцию на части данных 
    df = df.query('reg_ts >= Beginning and auth_ts <= End')
    df['number_days'] = ((df.auth_date - df.reg_date)/ np.timedelta64(1, 'D'))
    df_cohort = df.groupby(['reg_date', 'number_days']).agg({'uid':'nunique'}).reset_index()\
    .pivot_table(index='reg_date', columns = 'number_days', values = 'uid')
    cohort_size = cohort_pivot.iloc[:,0]
    retention_matrix = cohort_pivot.divide(cohort_size, axis =0)
    with sns.axes_style("white"):
        fig, ax=plt.subplots(1,2, figsize=(16,12),sharey=True, gridspec_kw={'width_ratios':[1,11]})
    #Тепловая карта для коээфициентов удержания
        sns.heatmap(retention_matrix,
               mask=retention_matrix.isnull(),
               annot=True,
               fmt='.0%',
               cmap='RdYlGn',
               ax=ax[1])
        ax[1].set_title('Daily Retention', fontsize=16)
        ax[1].set(xlabel='Количество дней после регистрации ', ylabel='Дата регистрации')
    #Тепловая карта для размеров когорт
        cohort_size_df = pd.DataFrame(cohort_size).rename(columns={0:'cohort_size'})
        white_cmap=mcolors.ListedColormap(['white'])
        sns.heatmap(cohort_size_df,
               annot=True,
               cbar= False,
               fmt='g',
               cmap=white_cmap,
               ax=ax[0])
        fig.tight_layout()
        plt.show()
    return sns.axes_style("white")

# Задание 2

Имеются результаты A/B теста, в котором двум группам пользователей предлагались различные наборы акционных предложений. Известно, что ARPU в тестовой группе выше на 5%, чем в контрольной. При этом в контрольной группе 1928 игроков из 202103 оказались платящими, а в тестовой – 1805 из 202667.

Какой набор предложений можно считать лучшим? Какие метрики стоит проанализировать для принятия правильного решения и как?

In [2]:
#Открываем файл 
df = pd.read_csv('Проект_1_Задание_2.csv', sep=';')
df

/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


,user_id,revenue,testgroup
0,1,0,b
1,2,0,a
2,3,0,a
3,4,0,b
4,5,0,b
...,...,...,...
404765,404766,0,a
404766,404767,0,b
404767,404768,231,a
404768,404769,0,a


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404770 entries, 0 to 404769
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    404770 non-null  int64 
 1   revenue    404770 non-null  int64 
 2   testgroup  404770 non-null  object
dtypes: int64(2), object(1)
memory usage: 9.3+ MB


In [5]:
df.nunique()

user_id      404770
revenue        1477
testgroup         2
dtype: int64

In [6]:
df.describe()

/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


,user_id,revenue
count,404770.000000,404770.000000
mean,202385.500000,26.083435
std,116847.178567,681.652928
min,1.000000,0.000000
25%,101193.250000,0.000000
50%,202385.500000,0.000000
75%,303577.750000,0.000000
max,404770.000000,37433.000000


In [7]:
#Высчитываем количество платящих пользователей для каждой группы
paying_users = df.query('revenue!=0').groupby(['testgroup']).agg(paying_users =('user_id','count'))
paying_users

,paying_users
testgroup,
a,1928
b,1805


Создаем отдельную таблицу с показателями отдельно для каждой группы. Рассчитываем:

Средний чек(ARPU) 

Количество пользователей(user_id) 

Количество платящих пользователей(paying_users) 

Доход(revenue)

Средний чек по платящему клиенту(ARPPU)

Конверсия в платящих пользователей(CR)

In [8]:
value = df.groupby(['testgroup']).agg({'revenue': 'sum', 'user_id':'count'}).reset_index()
value ['ARPPU'] = value['revenue']/value['user_id']
value ['ARPPU'] = value ['ARPPU'].round(2)
value = value.merge(paying_users, on='testgroup', how='right')
value ['ARPU'] = value['revenue']/value['paying_users']
value ['ARPU'] = value ['ARPU'].round(2)
value ['CR'] = (value['paying_users']/value['user_id'])*100
value

,testgroup,revenue,user_id,ARPPU,paying_users,ARPU,CR
0,a,5136189,202103,25.41,1928,2664.00,0.953969
1,b,5421603,202667,26.75,1805,3003.66,0.890624


По условию: в контрольной группе 1928 игроков из 202103 оказались платящими, а в тестовой – 1805 из 202667.

Значит, а - контрольная группа, b - тестовая группа.

Также видим, что показатели ARPPU и ARPU и доход больше в тестовой группе, но при этом конверсия и количество платящих пользователей больше в контрольной группе.

In [14]:
#Создаем отдельный столбец с показателем ARPU и совершил ли пользователь платеж или нет
df ['ARPU'] = (df['revenue']/df['user_id']).round(2)
df['Payment'] = df.revenue.apply(lambda x: x > 0)
df.head()

,user_id,revenue,testgroup,ARPPU,Payment,ARPU
0,1,0,b,0.0,False,0.0
1,2,0,a,0.0,False,0.0
2,3,0,a,0.0,False,0.0
3,4,0,b,0.0,False,0.0
4,5,0,b,0.0,False,0.0


In [15]:
#Создаем для каждой группы отдельную таблицу для удобства
df_a = df.query('testgroup=="a"')
df_b = df.query('testgroup=="b"')

Проверим гипотезу о взаимосвязи метрики ARPU между тестовой и контрольной группой. 

𝐻0 : взаимосвязи между переменными нет

𝐻1 : взаимосвязь есть

Для этого проверим является ли распределение переменной ARPU нормальной.

In [23]:
# Проверим нормальность распределения по Шапиро
pg.normality(data=df, dv="ARPU", group="testgroup")

/opt/tljh/user/lib/python3.7/site-packages/scipy/stats/morestats.py:1760: UserWarning:

p-value may not be accurate for N > 5000.

/opt/tljh/user/lib/python3.7/site-packages/scipy/stats/morestats.py:1760: UserWarning:

p-value may not be accurate for N > 5000.



,W,pval,normal
b,0.001494,0.0,False
a,0.000320,0.0,False


In [24]:
# по альтернативному тесту
pg.normality(data=df, dv="ARPU", group="testgroup", method="normaltest")

,W,pval,normal
b,1.094790e+06,0.0,False
a,1.218559e+06,0.0,False


Так как распределение не является нормальным, но при этом выборка очень большая для проверки гипотезы можно воспользоваться t-критерием с поправкой Уэлча.

In [16]:
st.ttest_ind(df_a.ARPU, df_b.ARPU, equal_var=False)

Ttest_indResult(statistic=0.3806473258449373, pvalue=0.7034652932290848)

Проверим гипотезу о взаимосвязи конверсии в платящих пользователей (CR) между тестовой и контрольной группой. Проверка на нормальность не требуется, т.к. проверяются категориальные переменные.

Так как переменная категориальная воспользуемся методом Хи-квадрат.

𝐻0 : взаимосвязи между переменными нет

𝐻1 : взаимосвязь есть

In [19]:
stat, p, dof, expected = chi2_contingency(pd.crosstab(df.Payment, df.testgroup))

In [20]:
stat, p

(4.374729521260405, 0.03647561892312613)

In [22]:
prob = 0.95
critical = chi2.ppf(prob, dof)
if abs(stat) >= critical:
    print('Отклоняем H0')
else:
    print('Не отклоняем H0')

Отклоняем H0


# Выводы

1. Так как p-value > 0.05 Нулевую гипотезу можно принять. Что означает, что взаимосвязи метрики ARPU между тестовой и контрольной группой нет.
2. Поскольку мы отклонили нулевую гипотезу можно предположить, что взаимосвязь конверсии в платящих пользователей (CR между тестовой и контрольной группой есть.

# Задание 3

В игре Plants & Gardens каждый месяц проводятся тематические события, ограниченные по времени. В них игроки могут получить уникальные предметы для сада и персонажей, дополнительные монеты или бонусы. Для получения награды требуется пройти ряд уровней за определенное время. С помощью каких метрик можно оценить результаты последнего прошедшего события?

Предположим, в другом событии мы усложнили механику событий так, что при каждой неудачной попытке выполнения уровня игрок будет откатываться на несколько уровней назад. Изменится ли набор метрик оценки результата? Если да, то как?

Для оценки результатов последнего прошедшего события можно рассмотреть  следующие метрики(так как в условиях не указано ничего о цене и платежах игроков во время события, финансовые метрики не берем в расчет):
1. DAU (Daily Active Users) — количество уникальных игроков за сутки;
2. Retention - какая доля игроков вернулась в продукт во время тематического события;
3. Churn rate — сколько игроков перестало играть за время тематического события;
4. CSAT (Customer Satisfaction Score)-  удовлетворенность игроков(можно провести опрос).

При усложнении механики событий набор метрик для оценки результата не изменится. Но также можно провести A/B тест и сравнить все написанные выше метрики.